## Alabama 2022 Primary Runoff Election Returns

### Sections
- <a href="#ETL">Cleaning Precinct-Level Election Results</a><br>
- <a href="#allocate">Allocate Absentee and Provisional Votes at Precinct Level</a><br>
- <a href="#check">Vote Totals Checks</a><br>
- <a href="#readme">Creating README</a><br>
- <a href="#exp">Exporting Cleaned Precinct-Level Dataset</a><br>

#### Sources

- [Alabama Primary Election Results, Precint Level](https://www.sos.alabama.gov/sites/default/files/election-data/2022-06/2022%20Primary%20Precinct%20Results.zip)
- [Secretary of State Certified Results, Democratic Party](https://www.sos.alabama.gov/sites/default/files/election-2022/AL%20Democratic%20Party%202022%20Primary%20Results.xlsx)
- [Secretary of State Certified Results, Republican Party](https://www.sos.alabama.gov/sites/default/files/election-2022/AL%20Republican%20Party%202022%20Primary%20Results%20Official.xlsx)
- [Secretary of State Certified Ballot Measure Results](https://www.sos.alabama.gov/sites/default/files/election-data/2022-11/Final%20Canvass%20of%20Results%20%28canvassed%20by%20state%20canvassing%20board%2011-28-2022%29.pdf)

In [1]:
import pandas as pd
import os
import numpy as np
import re
from collections import Counter
import AL_22_helper_functions as hlp

pd.set_option('display.max_rows', 1000)

In [2]:
# Temporarily ignore warning messages for vote allocation function
import warnings

warnings.filterwarnings("ignore")

<p><a name="ETL"></a></p>

### Cleaning Precinct Level Election Returns

Load-In + Clean Election Results

In [3]:
county_list = []
ph_county_list = []
contest_list = []
clean_index = []
index_issue = [] #use to 
files = os.listdir('./raw-from-source/2022 Primary Runoff Precinct Results/')

#list of words to filter for contests with statewide reach
contest_keywords = ['Senator', 'Representative', 'Governor', 'Attorney General', 'Secretary of State', 'Treasurer', 'Auditor', 'State Board of Education', 'BOE', 'Agriculture', 'Insurance', 'PSC', 'Public Service', 'Supreme Court', 'Court of Appeals', 'Amendment']

for idx, file in enumerate(files):
    #Load county files
    temp = pd.read_excel('./raw-from-source/2022 Primary Runoff Precinct Results/' + file)
    
    # Filter to contests of interest
    contest_keywords = set(keyword.upper() for keyword in contest_keywords)
    contest_titles_set = set(temp['Contest Title'])
    comb_list = [title for title in contest_titles_set if any(keyword in title for keyword in contest_keywords)]
    temp_statewide = temp[temp['Contest Title'].isin(comb_list)].copy()
    
    #Add to contest_list
    contest_list += list(temp_statewide['Contest Title'].unique())
    
    # Get the county name, clean "StClair" to match pattern
    county_name = file.split("-")[-1][0:-4]
    if county_name == "StClair":
        county_name = "St. Clair"
    county_list.append(county_name)
        
    # Clean the party name
    temp_statewide["Party"] = temp_statewide["Party"].str.strip()
    temp_statewide["Party"] = temp_statewide["Party"].fillna("")
    
    # Create a column to pivot on
    temp_statewide["pivot_col"] = temp_statewide["Contest Title"].str.strip()+"-:-"+temp_statewide["Candidate"].str.strip()
    temp_statewide["pivot_col"] = np.where(temp_statewide["Party"]=="",temp_statewide["pivot_col"],temp_statewide["pivot_col"]+"-:-"+temp_statewide["Party"].str.strip())
    
    # Drop columns that are no longer needed
    temp_statewide.drop(["Contest Title", "Party", "Candidate"], axis = 1, inplace = True)
    
    # Add the county name to the precinct
    rename_dict = {i:i+"-:-"+county_name for i in temp_statewide.columns if i != "pivot_col"}
    temp_statewide.rename(columns = rename_dict, inplace = True)
    
    # Transpose the dataframe
    temp_transpose = temp_statewide.set_index("pivot_col").T
    temp_transpose.reset_index(inplace = True, drop = False)
    
    # Make sure cols and indexes unique
    if temp_transpose.columns.nunique() == len(temp_transpose.columns) and temp_transpose.index.is_unique:
        clean_index.append(county_name)
    else:
        index_issue.append(str(county_name) + ' ' + str(idx))        

    # Add to the list of counties
    ph_county_list.append(temp_transpose)

In [4]:
# Check for number of unique contests
len(set(contest_list))

# Visually inspect contest types
#set(contest_list)

18

In [5]:
#check for index issues
len(index_issue)

0

In [6]:
# Concatenate into one file
comb = pd.concat(ph_county_list, axis = 0)

In [7]:
# Remove the under votes and the over votes
comb_list = [i for i in comb.columns.to_list() if "Under Votes" not in i and "Over Votes" not in i]
al_prim = comb[comb_list]

#### Rename Columns

In [8]:
# Create dictionaries for renaming pivot col to VEST
# use helper functions, found in helper module
exclude_columns = ['UNIQUE_ID', 'county', 'COUNTYFP', 'precinct', 'index']
contest_updates_dict, contest_updates_reversed, clean_dups =hlp.create_column_rename_dicts(al_prim, exclude_columns)

In [9]:
# Check all dict values under 10 characters
for item in contest_updates_dict.values():
    if len(item) > 10 or len(item) < 7:
        print(item)
        print(race_updates_reversed[item])

In [10]:
# Check 
len(contest_updates_dict) == al_prim.shape[1] - 1

True

In [11]:
# apply rename dictionary to df
al_prim.rename(columns = contest_updates_dict, inplace = True)
al_prim.reset_index(inplace = True, drop = True)

# Fillna
al_prim = al_prim.fillna(0)

# set columns with votes as integer type
for item in contest_updates_dict.values():
    al_prim[item] = al_prim[item].astype(int)

In [12]:
# Create DF of field names for later use in README creation
fieldnames = list(contest_updates_dict.values())
fieldnames.sort()

sorted_dict = dict(sorted(contest_updates_dict.items(), key=lambda x:x[1]))

export_dict = {i:key for key, i in sorted_dict.items()}
rm_df = pd.DataFrame(export_dict.items())

#### Add County, FIPS, Unique ID columns

In [13]:
# Define county, precinct columns
al_prim['county'] = al_prim['index'].apply(lambda x: x.split("-:-")[1])
al_prim['precinct'] = al_prim['index'].apply(lambda x: x.split("-:-")[0])

In [14]:
# Add FIPS col to precinct df
al_prim = hlp.create_fips_col("./raw-from-source/FIPS/US_FIPS_Codes.csv", 'Alabama', al_prim, 'county')

# Check
al_prim['COUNTYFP'].isnull().any()

False

In [15]:
# Create UNIQUE_ID col
al_prim['UNIQUE_ID'] = al_prim['COUNTYFP'] + '-' +al_prim['precinct']
# Check
al_prim['UNIQUE_ID'].nunique()

2070

In [16]:
# Filter down to needed columns
al_prim = al_prim[['UNIQUE_ID', 'COUNTYFP', 'county', 'precinct']+list(contest_updates_dict.values())]

In [17]:
al_prim.head(2)

pivot_col,UNIQUE_ID,COUNTYFP,county,precinct,R22USSRBRI,R22USSRBRO,R22GOVDFLO,R22GOVDFOR,R22SOSRALL,R22SOSRZEI,...,RSL057DSEL,RSL057DWIN,RSL002RBLA,RSL002RHAR,RSL004RJOH,RSL004RMOO,RSL020RLOM,RSL020RTAY,RSL100RKUP,RSL100RSHI
0,001-10 JONES COMM_ CTR_,001,Autauga,10 JONES COMM_ CTR_,47,34,5,26,45,36,...,0,0,0,0,0,0,0,0,0,0
1,001-100 TRINITY METHODIST,001,Autauga,100 TRINITY METHODIST,407,259,4,3,416,234,...,0,0,0,0,0,0,0,0,0,0


<p><a name="allocate"></a></p>

### Allocate Absentee and Provisional Votes at the Precinct Level

In [18]:
absentee_precs = al_prim[(al_prim["UNIQUE_ID"].str.contains("ABSENTEE"))|(al_prim["UNIQUE_ID"].str.contains("PROVISIONAL"))|(al_prim["UNIQUE_ID"].str.contains("PROVISONAL"))]
nonabsentee_precs = al_prim[~((al_prim["UNIQUE_ID"].str.contains("ABSENTEE"))|(al_prim["UNIQUE_ID"].str.contains("PROVISIONAL"))|(al_prim["UNIQUE_ID"].str.contains("PROVISONAL")))]

print(absentee_precs.shape)
print(nonabsentee_precs.shape)

(135, 40)
(1935, 40)


In [19]:
# Check for anomalies
#absentee_precs['county'].value_counts()

Lamar county has a typo in their 'provisional' votes precinct name.
Jefferson county has two categories of absentee votes

#### Jefferson county vote allocation

Jefferson county vote allocation will be done separately because provisional votes are reported at the county level, but absentee votes are reported in two categories of Birmingham precincts and Bessemer precincts.
information on categorizing precincts in this county was found on the [county website](https://www.jccal.org/Default.asp?ID=1936&pg=Maps)

In [20]:
# Create a list of all precincts in jefferson county
jco = al_prim[al_prim['county'] == 'Jefferson']['UNIQUE_ID'].to_list()
jco = jco[2:]

In [21]:
# use list to subset to those precincts indicated as belonging to bessemer
bessemer_precs = [jco[4], jco[8], jco[22], jco[25], jco[29],
                  jco[36], jco[38], jco[40], jco[46], jco[52],
                  jco[53], jco[55], jco[56], jco[62], jco[63],
                  jco[66], jco[67], jco[77], jco[78], jco[79],
                  jco[80], jco[81], jco[83], jco[84], jco[85],
                  jco[89], jco[90], jco[91], jco[92], jco[93],
                 jco[94], jco[95], jco[98], jco[99], jco[101],
                 jco[103], jco[104], jco[105], jco[106], jco[109],
                 jco[110], jco[111], jco[112], jco[113], jco[116],
                 jco[117], jco[118], jco[119], jco[120], jco[148],
                 jco[150], jco[158]]

In [22]:
# Check that all precincts are allocated to bessemer or birmingham
birmingham_precs = list(set(jco) - set(bessemer_precs))
print('Number of Birmingham Precincts: ', len(birmingham_precs))
print('Number of Bessemer Precincts: ',len(bessemer_precs),)
print('Total Precincts: ', len(jco))

Number of Birmingham Precincts:  123
Number of Bessemer Precincts:  52
Total Precincts:  175


In [23]:
# Visually check precinct names against state PDFS
#bessemer_precs
#birmingham_precs

In [24]:
# Filter out the absentee precincts related to Birmingham or Bessemer
jefferson_exceptions = absentee_precs[absentee_precs["UNIQUE_ID"].str.contains("BIRMINGHAM") | absentee_precs["UNIQUE_ID"].str.contains("BESSEMER")]

# Make a list of the remaining absentee precincts
remaining_absentee =  absentee_precs[~(absentee_precs["UNIQUE_ID"].str.contains("BIRMINGHAM") | absentee_precs["UNIQUE_ID"].str.contains("BESSEMER"))]

In [25]:
# Going to use the names "BESSEMER" and "BIRMINGAHM" as the keys to allocate on and not the county names
jefferson_exceptions["Spec_Alloc"] = jefferson_exceptions["precinct"].apply(lambda x: x.split( )[0])

birmingham_precincts = nonabsentee_precs[(~nonabsentee_precs["UNIQUE_ID"].isin(bessemer_precs)) & (nonabsentee_precs["county"]=="Jefferson")]
bessemer_precincts = nonabsentee_precs[nonabsentee_precs["UNIQUE_ID"].isin(bessemer_precs)]

birmingham_precincts["Spec_Alloc"] = "BIRMINGHAM"
bessemer_precincts["Spec_Alloc"] = "BESSEMER"

In [26]:
# combine into one df
receiving_jefferson = pd.concat([birmingham_precincts, bessemer_precincts])

In [27]:
# Get the precincts that need votes in the rest of the state
non_jefferson_non_absentee = nonabsentee_precs[~nonabsentee_precs["UNIQUE_ID"].isin(list(receiving_jefferson["UNIQUE_ID"]))]

In [28]:
# columns that need allocation
vote_cols = al_prim.columns[4:]

In [29]:
# Run vote allocation for jefferson county
received_jefferson = hlp.allocate_absentee(receiving_jefferson, jefferson_exceptions, list(vote_cols), "Spec_Alloc", allocating_to_all_empty_precs=False)

Special allocation used for [['BIRMINGHAM', 'RSL056DHUF']]


In [30]:
received_jefferson.drop("Spec_Alloc", axis = 1, inplace = True)

#### Absentee Allocation for non Jefferson counties

In [31]:
round_two = pd.concat([received_jefferson, non_jefferson_non_absentee])

In [32]:
round_two.reset_index(inplace = True, drop = True)
remaining_absentee.reset_index(inplace = True, drop = True)

In [33]:
# run final vote allocation for 66 non jefferson counties
final_allocation = hlp.allocate_absentee(round_two, remaining_absentee, list(vote_cols), "county", allocating_to_all_empty_precs=False)

<p><a name="check"></a></p>

### Vote Totals Check - Statewide

#### Republican

In [34]:
# Read in edited summary sheet
df = pd.read_excel('./raw-from-source/AL Republican Party 2022 Runoff Results Official.xlsx', sheet_name='summary_edited')
# Extract columns with vote total information
df1 = df[df.columns[:3]]
df2 = df[df.columns[4:7]]
df3 = df[df.columns[8:11]]
# create column names
df1.columns = df2.columns = df3.columns = ['contest', 'choice', 'num_votes']
# concatenate into one df
sos_combined = pd.concat([df1, df2, df3], axis=0)

In [35]:
#check
df1.shape[0] + df2.shape[0] + df3.shape[0] == sos_combined.shape[0]

True

In [36]:
#filter out null rows
sos_r = sos_combined[~sos_combined['contest'].isnull()]
#set num_votes column as integer
sos_r.num_votes = sos_r.num_votes.astype(int)
sos_r.num_votes.dtype

dtype('int32')

In [37]:
#account for anomalies in contest names
sos_r['contest'] = sos_r['contest'].replace('Govenor', 'Governor')
sos_r['party'] = 'REP'

In [38]:
sos_r_p = hlp.create_pivot_col(sos_r, 'choice', 'contest', 'party', 'pivot').copy()

In [39]:
# Create rename dict to read in county-level results
sos_r_dict = dict(zip(sos_r_p['choice'], sos_r_p['contest']))

In [40]:
# Create placeholder dataframe
sos_r_county = pd.DataFrame(columns=['choice', 'num_votes', 'county', 'contest'])

In [41]:
# extract data fom each county sheet, process and add to new dataframe
counties_worked = []
county_list_subset = county_list[:1]
sos_r_county = pd.DataFrame()
for county in county_list:
    df = pd.read_excel('./raw-from-source/AL Republican Party 2022 Runoff Results Official.xlsx', sheet_name= str(county))
    # Extract columns with vote total information
    df1 = df[df.columns[:2]]
    df2 = df[df.columns[4:6]]
    # create column names
    df1.columns = df2.columns = ['choice', 'num_votes']
    # concatenate into one df
    county_single = pd.concat([df1, df2], axis=0)
    # drop rows that are empty in choice and vote columns
    county_single_filt = county_single.dropna(subset=['choice', 'num_votes'], how = 'all')
    # remove '.', ',' to match to sos_r_dict
    county_single_filt['choice'] = county_single_filt.astype(str)
    county_single_filt['choice'] = county_single_filt['choice'].apply(lambda x: x.replace('.', ''))
    county_single_filt['choice'] = county_single_filt['choice'].apply(lambda x: x.replace(',', ''))
    # use sos_r_dict to match candidate name to contest
    county_single_filt['contest'] = county_single_filt['choice'].map(sos_r_dict)
    # drop rows without contest
    county_single_filt = county_single_filt.dropna(subset=['contest'])
    # add county name column
    county_single_filt['county'] = str(county)
    # add county name to a list of counties
    counties_worked.append(county)
    # add dataframe to receiving df
    sos_r_county = pd.concat([sos_r_county, county_single_filt], ignore_index=True)

In [42]:
len(counties_worked)

67

In [43]:
# check that all counties worked
len(counties_worked)
# Check number of counties in new df
sos_r_county['county'].nunique()

67

In [44]:
# add in party column, fill null votes with 0
sos_r_county['num_votes'].fillna(0, inplace=True)
sos_r_county['party'] = 'REP'

In [45]:
# add 'pivot' col
# use helper function
sos_r_pvt = hlp.create_pivot_col(sos_r_county, 'choice', 'contest', 'party', 'pivot')
# use pivot col to add column with VEST names
sos_r_pvt['VEST'] = sos_r_pvt['pivot'].apply(lambda x: hlp.get_VEST(str(x).strip()))

In [46]:
#pivot df
sos_r_pvt =pd.pivot_table(sos_r_pvt,index=['county'],columns=['VEST'],values=['num_votes'],aggfunc=sum)
sos_r_pvt = sos_r_pvt.fillna(0)
sos_r_pvt.columns = sos_r_pvt.columns.droplevel(0)
sos_r_pvt.reset_index(inplace = True)

In [47]:
sos_r_pvt.head(2)

VEST,county,R22AUDRCOO,R22AUDRSOR,R22PS1RODE,R22PS1RWOO,R22PS2RBEE,R22PS2RMCC,R22SOSRALL,R22SOSRZEI,R22USSRBRI,...,RSL014RFRE,RSL014RWAD,RSL020RLOM,RSL020RTAY,RSL040RBOR,RSL040RROB,RSL100RKUP,RSL100RSHI,RSU12RDRA,RSU12RKEL
0,Autauga,1784.0,3180.0,2621.0,2024.0,2965.0,1736.0,3432.0,1814.0,3414.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Baldwin,6942.0,12297.0,8094.0,9676.0,9158.0,8629.0,10329.0,10381.0,14943.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Democratic Election Results from SOS

In [48]:
# Read in results
df = pd.read_excel('./raw-from-source/2022 Democratic Runoff Results.xlsx')
sos_d = df.fillna(0)

In [49]:
# select columns
sos_d = sos_d[sos_d.columns[:-1]]

In [50]:
sos_d.head(2)

,County,R22GOVDFOR,R22GOVDFLO,RSU23DSAN,RSU23DSTE,RSL055DPLU,RSL055DSCO,RSL056DHUF,RSL056DTIL,RSL057DSEL,RSL057DWIN
0,Autauga,204,166,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Baldwin,284,244,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Combine Republican and Democratic Results

In [51]:
sos_tot_pvt = pd.concat([sos_r_pvt, sos_d[sos_d.columns[1:]]], axis= 1)

In [52]:
#check
sos_r_pvt.shape[0] == sos_d.shape[0] == sos_tot_pvt.shape[0]

True

In [53]:
# Check to see if the number of contests is the same
print('# of contests in SOS df:', len(sos_tot_pvt.columns))
print('# of contests in RDH df:', len(vote_cols))

# of contests in SOS df: 37
# of contests in RDH df: 36


In [54]:
#Check which contests missing in SOS combined df
set(vote_cols) == set(sos_tot_pvt.columns)
set(vote_cols) - set(sos_tot_pvt.columns)

set()

### Statewide Check

In [55]:
# Check statewide totals for each contest
statewide_check_list = []
doesnt_check = []
for item in vote_cols:
    if item not in sos_tot_pvt.columns:
        doesnt_check.append(item)
#         print(item)
#         print(contest_updates_reversed[item])
    else:
        official = sos_tot_pvt[item].sum().astype(int)
    rdh = final_allocation[item].sum()
    if official != rdh:
        statewide_check_list.append(item)
        print(contest_updates_reversed[item])
        print('')
        print(f"{item}\n\tOfficial: {official}\n\tRDH: {rdh}")

UNITED STATES SENATOR-:-Katie Britt-:-REP

R22USSRBRI
	Official: 253251
	RDH: 253250
UNITED STATES SENATOR-:-Mo Brooks-:-REP

R22USSRBRO
	Official: 148636
	RDH: 148637
SECRETARY OF STATE-:-Wes Allen-:-REP

R22SOSRALL
	Official: 248132
	RDH: 248130
STATE AUDITOR-:-Stan Cooke-:-REP

R22AUDRCOO
	Official: 152006
	RDH: 152007
PUBLIC SERVICE COMMISSION, PLACE 1-:-Brent Woodall-:-REP

R22PS1RWOO
	Official: 160263
	RDH: 160264
PUBLIC SERVICE COMMISSION, PLACE 2-:-Chip Beeker-:-REP

R22PS2RBEE
	Official: 215377
	RDH: 215379
PUBLIC SERVICE COMMISSION, PLACE 2-:-Robert L. McCollum-:-REP

R22PS2RMCC
	Official: 125178
	RDH: 125179
STATE REPRESENTATIVE, DISTRICT 55-:-Fred "Coach" Plump-:-DEM

RSL055DPLU
	Official: 952
	RDH: 948
STATE REPRESENTATIVE, DISTRICT 55-:-Roderick "Rod" Scott-:-DEM

RSL055DSCO
	Official: 919
	RDH: 915
STATE REPRESENTATIVE, DISTRICT 20-:-James Lomax-:-REP

RSL020RLOM
	Official: 5385
	RDH: 5384


In [56]:
len(statewide_check_list)

10

In [57]:
#county totals check
rdh = al_prim
sos = sos_tot_pvt
partner_name = 'SOS'
source_name = 'RDH'
county_col = 'county'
hlp.county_totals_check(sos,partner_name, rdh, source_name, sos_tot_pvt.columns[1:], county_col,full_print=False, method='county')

***Countywide Totals Check***

Autauga contains differences in these races:
	R22USSRBRI has a difference of 1.0 vote(s)
		SOS: 3414.0 vote(s)
		RDH: 3413 vote(s)
Jefferson contains differences in these races:
	RSL055DPLU has a difference of 4.0 vote(s)
		SOS: 952.0 vote(s)
		RDH: 948 vote(s)
	RSL055DSCO has a difference of 4.0 vote(s)
		SOS: 919.0 vote(s)
		RDH: 915 vote(s)
Lawrence contains differences in these races:
	R22SOSRALL has a difference of 3.0 vote(s)
		SOS: 1979.0 vote(s)
		RDH: 1976 vote(s)
Madison contains differences in these races:
	RSL020RLOM has a difference of 1.0 vote(s)
		SOS: 5385.0 vote(s)
		RDH: 5384 vote(s)
Mobile contains differences in these races:
	R22AUDRCOO has a difference of -1.0 vote(s)
		SOS: 7704.0 vote(s)
		RDH: 7705 vote(s)
	R22PS1RWOO has a difference of -1.0 vote(s)
		SOS: 11116.0 vote(s)
		RDH: 11117 vote(s)
	R22PS2RBEE has a difference of -1.0 vote(s)
		SOS: 9963.0 vote(s)
		RDH: 9964 vote(s)
	R22SOSRALL has a difference of -1.0 vote(s)
		SOS: 1

<p><a name="discrep"></a></p>

### Addressing Discrepancies in Vote Total Checks

### Summary

Counties w/ unaccounted for vote total discrepancies : Autauga, Jefferson, Lawrence, Madison, Mobile, Perry

###### Accounted for discrepancies:
- RDH 'undercount' compared to SOS
 1. Autauga - RDH has -1 vote
         - R22PS2RMCC
 2. Jefferson - RDH has -4 votes
         - RSL055DPLU
         - RSL055DSCO, 
 3. Lawrence - RDH has -3 votes
         - R22SOSRALL
 4. Madison - RDH has -1 vote
         - RSL020RLOM
 
- RDH 'overcount' compared to SOS
1. Mobile - RDH has +1 vote for the following candidates
        - R22AUDRCOO
        - R22PS1RWOO
        - R22PS2RBEE
        - R22SOSRALL
        - R22USSRBRO
2. Perry - RDH has +1 vote for the following candidates
        - R22PS2RBEE
        - R22PS2RMCC


<p><a name="readme"></a></p>

### Creating README

In [58]:
#rename columns
rm_df.columns = ['VEST', 'PIVOT']

In [59]:
# split pivot column into standardized contest, party, rm_name
rm_df['contest'] = rm_df['PIVOT'].apply(lambda x : x.split('-:-')[0])
rm_df['name'] = rm_df['PIVOT'].apply(lambda x: x.split('-:-')[1])
rm_df['party'] = rm_df['PIVOT'].apply(lambda x : x.split('-:-')[-1])
rm_df['contest_stnd'] = rm_df['contest'] + ' - ' + rm_df['party']
rm_df['rm_name'] =rm_df['name'].apply(lambda x: x.split()[-1] + ', ' + ' '.join(x.split()[:-1]))

In [60]:
contests_unord = rm_df['contest'].unique()
#contests_unord

In [61]:
# make list of contest types
undist = ['UNITED STATES SENATOR',
          'GOVERNOR',
          'SECRETARY OF STATE',
          'STATE AUDITOR']
psc = ['PUBLIC SERVICE COMMISSION, PLACE 1',
       'PUBLIC SERVICE COMMISSION, PLACE 2']
us_cong = ['UNITED STATES REPRESENTATIVE, 5TH CONGRESSIONAL DISTRICT']
al_senate = ['STATE SENATOR, DISTRICT 12',
       'STATE SENATOR, DISTRICT 23']
al_house = [ 'STATE REPRESENTATIVE, DISTRICT 2',
       'STATE REPRESENTATIVE, DISTRICT 4',
       'STATE REPRESENTATIVE, DISTRICT 14',
       'STATE REPRESENTATIVE, DISTRICT 20',
       'STATE REPRESENTATIVE, DISTRICT 40',
       'STATE REPRESENTATIVE, DISTRICT 55',
       'STATE REPRESENTATIVE, DISTRICT 56',
       'STATE REPRESENTATIVE, DISTRICT 57',
       'STATE REPRESENTATIVE, DISTRICT 100']
contests_ord = undist + psc  + us_cong + al_senate + al_house

In [62]:
#check
set(contests_ord) == set(contests_unord)

True

In [63]:
# Create list of ordered stnd contests for all contest types
contests_order =[]
for i in contests_ord:
    temp_df = rm_df.loc[rm_df['contest'] == i]
    contests_sorted = sorted(temp_df['contest_stnd'].unique().tolist())
    contests_order += contests_sorted

In [64]:
# adda total vote column to determine vote share
rm_df['vote_tot'] = rm_df['VEST'].apply(lambda x: sum(final_allocation[x]))

In [65]:
# create list to order README and final dataset by contest, and vote share
rm_order =[]
for i in contests_order:
    temp = rm_df.loc[rm_df['contest_stnd'] == i].sort_values('vote_tot', ascending = False)
    rm_order += temp['VEST'].to_list()
#check to make sure all contests included
set(rm_order) == set(vote_cols)

True

In [66]:
#Create order column, mapping ordering list to VEST names
rm_df['Order'] = rm_df['VEST'].map(lambda x: rm_order.index(x))
#order the readme DF
rm_df = rm_df.sort_values('Order')
#create field name column
rm_df['description'] = rm_df['rm_name'] + ' - ' + rm_df['contest_stnd']

In [67]:
#create fields_dict
fields_dict = dict(zip(rm_df['VEST'], rm_df['description']))

In [68]:
### Create README

fields_dict['UNIQUE_ID']='Unique ID for each precinct'
fields_dict['COUNTYFP']='County FIP identifier'
fields_dict['county']='County Name'
fields_dict['precinct']='Precinct Name'

title = "Alabama 2022 Primary Runoff Election Precinct-Level Results"
retrieval_date = "10/05/23"
fields_dict = fields_dict
github_link = "https://github.com/nonpartisan-redistricting-datahub/pber_collection/tree/main/AL"
file_folder = "./"
source = "Alabama Secretary of State"

In [69]:
def full_readme_text(title, retrieval_date, source, fields_dict, github_link):

#First section of README
    readme_p1 = '''{title}\n
## RDH Date Retrieval
{retrieval_date}

## Sources
{source}

## Notes on Field Names (adapted from VEST):
Columns reporting votes generally follow the pattern: 
One example is:
G16PREDCLI
The first character is G for a general election, P for a primary, S for a special, and R for a runoff.
Characters 2 and 3 are the year of the election.*
Characters 4-6 represent the office type (see list below).
Character 7 represents the party of the candidate.
Characters 8-10 are the first three letters of the candidate's last name.

*To fit within the GIS 10 character limit for field names, the naming convention is slightly different for the State Legislature, Public Service Commissioners and US House of Representatives. All fields are listed below with definitions.

Office Codes Used:
AUD - State Auditor
GOV - Governor
PS# - Public Service Commissioner
SOS - Secretary Of State
USS - United States Senator
CON## - U.S. Congress
SL###  - State Legislative Lower
SU##  - State Legislative Upper

## Fields:
'''.format(title = title, source = source, retrieval_date = retrieval_date)

#Second section of README
    fields_table = pd.DataFrame.from_dict(fields_dict.items())
    fields_table.columns = ["Field Name", "Description"]
    readme_p2 = fields_table.to_string(formatters={'Description':'{{:<{}s}}'.format(fields_table['Description'].str.len().max()).format, 'Field Name':'{{:<{}s}}'.format(fields_table['Field Name'].str.len().max()).format}, index=False, justify = "left")

#Third section of README
    readme_p3 = '''\n
## Processing Steps
Visit the RDH GitHub and the processing script for this code [here]({github_link})

## Additional Notes

Files were checked against republican and democratic offician election results files also available from the Alabama Secretary of State. Results matches exactly except for in the instances below.


For the following counties and candidates, we were unable to account for vote total discrepancies between the precinct-level results and the state-level results.
In some instances, we believe the discrepancy is likely due to a data entry error or typo. Since we are unable to determine if each typo happened at the county or state level, we have chosen to leave the precinct-level results unaltered in the following cases.
1. Autauga - RDH has -1 vote
         - R22PS2RMCC
2. Jefferson - RDH has -4 votes
         - RSL055DPLU
         - RSL055DSCO, 
3. Lawrence - RDH has -3 votes
         - R22SOSRALL
4. Madison - RDH has -1 vote
         - RSL020RLOM
5. Mobile - RDH has +1 vote for the following candidates
        - R22AUDRCOO
        - R22PS1RWOO
        - R22PS2RBEE
        - R22SOSRALL
        - R22USSRBRO
6. Perry - RDH has +1 vote for the following candidates
        - R22PS2RBEE
        - R22PS2RMCC

Please direct questions related to processing this dataset to info@redistrictingdatahub.org.
'''.format(github_link=github_link)
    
    full_readme = str(readme_p1)+str(readme_p2)+str(readme_p3)
    return full_readme

In [70]:
if not os.path.exists(file_folder):
    os.mkdir(file_folder)

with open(file_folder+"README.txt", 'w') as tf:
        tf.write(full_readme_text(title, retrieval_date, source, fields_dict, github_link))

<p><a name="exp"></a></p>

### Exporting Cleaned Precinct-Level Dataset

In [71]:
rm_order = ['UNIQUE_ID', 'COUNTYFP', 'county', 'precinct'] + rm_order

In [72]:
#checks
len(rm_order) == len(final_allocation.columns)
set(rm_order) == set(final_allocation.columns)

True

In [73]:
#reorder df
final_allocation = final_allocation[rm_order]

In [74]:
if not os.path.exists("./al_2022_prim_runoff_prec/"):
    os.mkdir("./al_2022_prim_runoff_prec/")

final_allocation.to_csv("./al_2022_prim_runoff_prec/al_2022_prim_runoff_prec.csv", index = False)

In [75]:
final_allocation.head()

pivot_col,UNIQUE_ID,COUNTYFP,county,precinct,R22USSRBRI,R22USSRBRO,R22GOVDFLO,R22GOVDFOR,R22SOSRALL,R22SOSRZEI,...,RSL040RROB,RSL040RBOR,RSL055DPLU,RSL055DSCO,RSL056DTIL,RSL056DHUF,RSL057DSEL,RSL057DWIN,RSL100RSHI,RSL100RKUP
0,073-PREC 1010 - HUFFMAN BAPTIST CH,073,Jefferson,PREC 1010 - HUFFMAN BAPTIST CH,100,120,147,77,140,66,...,0,0,0,0,0,0,0,0,0,0
1,073-PREC 1020 - TOM BRADFORD PARK,073,Jefferson,PREC 1020 - TOM BRADFORD PARK,80,100,90,50,109,69,...,0,0,0,0,0,0,0,0,0,0
2,073-PREC 1030 - L_M_ SMITH MIDDLE,073,Jefferson,PREC 1030 - L_M_ SMITH MIDDLE,26,27,103,45,35,16,...,0,0,0,0,0,0,0,0,0,0
3,073-PREC 1040 - BETHEL BAPTIST CHU,073,Jefferson,PREC 1040 - BETHEL BAPTIST CHU,5,5,149,64,4,5,...,0,0,0,0,0,0,0,0,0,0
4,073-PREC 1060 - SUN VALLEY ELEMENT,073,Jefferson,PREC 1060 - SUN VALLEY ELEMENT,26,44,117,54,48,20,...,0,0,0,0,0,0,0,0,0,0
